## Summing all the prime numbers below a given number

In [1]:
import sys
#parámetro desde la línea de comandos
number=2_500_000

if len(sys.argv) > 1: #Usa el último argumento de la línea si es un entero
    last = sys.argv[-1]
    if last.isdigit():
        number = int(last)

print(f"El valor de 'number' es: {number}")

El valor de 'number' es: 2500000


In [1]:
import time

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 4.78333044052124
The prime sum below  2500000 is  219697708195  and the time taken is 4.57 s ± 1.14 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


Apartado a) Aplicando el paquete Numba

In [2]:
from numba import njit


# Simple code

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 0.6653797626495361
The prime sum below  2500000 is  219697708195  and the time taken is 182 ms ± 162 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


Apartado b) Usando Multiprocessing con Pool

In [3]:
import time
from numba import njit
from multiprocessing import Pool

# Simple code

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

def sum_primes_interval(args): #función worker, necesaria para ejecución en paralelo
    a, b = args
    result=0
    for i in range(a,b):
        result += if_prime(i)
    return result

def sum_primes_parallel(number, ncores): #funcion orquestadora que divide el trabajo entre los 4 núcleos
    step= number// ncores
    intervals= [ #lista de tuplas
    (0,step),
    (step, 2*step),
    (2*step, 3*step),
    (3*step, number)
]
    with Pool(processes=ncores) as p:
        partial_sums=p.map(sum_primes_interval, intervals) #cada proceso recibe una tupla y solo suma en ese rango
    return sum(partial_sums)
        

suma = 0
N = 3 # number of loops
ncores=4

start = time.time()
for i in range(N):
    suma=sum_primes_parallel(number, ncores)
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes_parallel(number,ncores)
suma = sum_primes_parallel(number,ncores)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 0.267730712890625
The prime sum below  2500000 is  219697708195  and the time taken is 261 ms ± 557 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


Apartado c) Usando Numba con prange

In [4]:
from numba import njit, prange, set_num_threads, get_num_threads


# Simple code

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

@njit(parallel=True) #creamos una función para paralelizar con prange de Numba
def sum_primes_prange(x):
    result=0
    for i in prange(x):
        result += if_prime(i)
    return result
        

number = 2_500_000
suma = 0
N = 3 # number of loops

set_num_threads(4) #para usar 4 núcleos
print("Numba threads:", get_num_threads())

start = time.time()
for i in range(N):
    suma = sum_primes_prange(number)
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes_prange(number)
suma = sum_primes_prange(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


Numba threads: 4
The prime sum below  2500000 is  None  and the time taken is 0.23790105183919272
The prime sum below  2500000 is  None  and the time taken is 70 ms ± 22.9 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)


CONCLUSIÓN DE LOS 4 ENFOQUES ESTUDIA

Aunque los tiempos medidos dentro de Python permiten comparar el rendimiento relativo de los distintos métodos, los tiempos obtenidos a través de SLURM reflejan el coste real de ejecución en el clúster siendo por tanto los más representativos en un entorno HPC.
Los tiempos obtenidos en cada caso muestran las diferencias entre cada enfoque usado. El código original en Python presenta el tiempo de ejecución más elevado (8s) al ser el código primitivo y no optimizado. Aplicando el paquete Numba (@njit) sobre el código anterior, el tiempo se reduce considerablemente al compilar las funciones a código maquina, sin embargo, sigue siendo código secuencial, por lo que se usa un único núcleo. Las mejoras más notables se observan al paralelizar el programa usando multiprocessing y Numba con prange. En el primero de estos casos el tiempo se ve reducido al dividir el trabajo entre varios procesos, sin embargo, el tiempo es mayor que al usar Numba con prange, esto es debido a que existe un coste extra de la gestión y creación de los procesos. Por último, usando prange de Numba, el tiempo de ejecución se ha visto reducido considerablemente debido a que ahora se paraleliza usando 4 hilos, pero no se crea ningún proceso nuevo, por lo que se evita el coste que aumentaba el tiempo cuando usabamos multiprocessing.